## &#10148; Requried Libraries</br>

In [4]:
import pandas as pd
import json
import csv
import gzip
import itertools
import string
import re
import wordcloud
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import pylab as pl
from textblob import TextBlob
import spacy                                                  # Import spaCy library
from spacy.lang.en import English                             # Import specific model
nlp = spacy.load("en_core_web_sm")
import warnings                                                  #To avoid the shown warnings
warnings.filterwarnings("ignore")

In [3]:
reviews = pd.read_json('Musical_Instruments.json.gz', lines=True)
metadata = pd.read_json('meta_Musical_Instruments.json.gz', lines=True)

In [5]:
reviews.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,image
0,5,90,False,"08 9, 2004",AXHY24HWOF184,0470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600,NaN
1,4,2,True,"04 6, 2017",A29OWR79AM796H,0470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,1491436800,NaN
2,5,NaN,True,"03 14, 2017",AUPWU27A7X5F6,0470536454,{'Format:': ' Paperback'},Amazon Customer,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,Five Stars,1489449600,NaN
3,4,NaN,True,"02 14, 2017",A1N69A47D4JO6K,0470536454,{'Format:': ' Paperback'},Christopher Burnett,My girlfriend use quite often,Four Stars,1487030400,NaN
4,5,NaN,True,"01 29, 2017",AHTIQUMVCGBFJ,0470536454,{'Format:': ' Paperback'},Amazon Customer,Arrived as described. Very happy.,Very happy.,1485648000,NaN


In [6]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Musical Instruments, Drums & Percussion, Hand...",,[Cricket Rubbing the spine with the wooden sti...,,Wooden Percussion 2 Piece Set of 3 Inch Cricke...,"[B00NP8GYVS, B00NP80XMO, B00NP8M098]",,WADSUWAN SHOP,"[Wood percussion, Owl whistle*, Includes woode...","[>#141,729 in Musical Instruments (See Top 100...",[],Musical Instruments,,"December 2, 2013",,0000989983,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Musical Instruments, Drums & Percussion, Hand...",,[Frog - Rubbing its spine with the wooden stic...,,"Wooden Percussion 3 Piece Set Frog, Cricket an...","[B00NP8GYVS, B00NP80XMO, B01MY48HK5, B00AZZ1AJ...",,WADSUWAN SHOP,"[Wood percussion, Small 3 inches, Creates orig...","[>#1,622 in Musical Instruments (See Top 100 i...",[],Musical Instruments,,"December 2, 2013",$0.91,0000098906,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Musical Instruments, Instrument Accessories, ...",,[Vivaldi's famous set of four violin concertos...,,Hal Leonard Vivaldi Four Seasons for Piano (Or...,[],,Hal Leonard,"[., ., .]","[>#330,653 in Musical Instruments (See Top 100...",[],Musical Instruments,,"May 10, 2011",$62.93,0041291905,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,[],,"[The Turn of the Screw (op. 54) vocal score, p...",,The Turn of the Screw (vocal score),"[0486266842, 0793507669, 0393008789, 142341280...",,Boosey &amp; Hawkes,[],"[>#86,354 in Musical Instruments (See Top 100 ...",[],Musical Instruments,,"May 23, 2007",$107.79,0060015500,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,[],,[],,Suite for Organ (including the Trumpet Volunta...,[],,,[],"[>#482,025 in Musical Instruments (See Top 100...",[],Musical Instruments,,"February 8, 2013",,0193757710,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


- Shape of review data
- Columns of review data
- Statistical evualtion of the review data
- Data type of review data

In [7]:
print('Shape of review data:\n',reviews.shape)
print('\n')
print('#'*100,'\n')
print('Columns of review data:\n ',reviews.columns)
print('\n')
print('#'*100,'\n')
print('Statistical evualtion of the review data:\n',reviews.describe())
print('\n')
print('#'*100,'\n')
print('Data type of review data:\n',reviews.dtypes)

Shape of review data:
 (1512530, 12)


#################################################################################################### 

Columns of review data:
  Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image'],
      dtype='object')


#################################################################################################### 

Statistical evualtion of the review data:
             overall  unixReviewTime
count  1.512530e+06    1.512530e+06
mean   4.257825e+00    1.436846e+09
std    1.238454e+00    6.587492e+07
min    1.000000e+00    9.470304e+08
25%    4.000000e+00    1.404346e+09
50%    5.000000e+00    1.447805e+09
75%    5.000000e+00    1.484525e+09
max    5.000000e+00    1.538611e+09


#################################################################################################### 

Data type of review data:
 overall            int64
vote           

In [8]:
metadata.shape

(120310, 19)

- Shape of meta data
- Columns of meta data
- Statistical evualtion of the meta data
- Data type of meta data

In [9]:
print('Shape of meta data:\n',metadata.shape)
print('\n')
print('#'*100,'\n')
print('Columns of meta data:\n ',metadata.columns)
print('\n')
print('#'*100,'\n')
print('Data type of meta data:\n',metadata.dtypes)

Shape of meta data:
 (120310, 19)


#################################################################################################### 

Columns of meta data:
  Index(['category', 'tech1', 'description', 'fit', 'title', 'also_buy', 'tech2',
       'brand', 'feature', 'rank', 'also_view', 'main_cat', 'similar_item',
       'date', 'price', 'asin', 'imageURL', 'imageURLHighRes', 'details'],
      dtype='object')


#################################################################################################### 

Data type of meta data:
 category           object
tech1              object
description        object
fit                object
title              object
also_buy           object
tech2              object
brand              object
feature            object
rank               object
also_view          object
main_cat           object
similar_item       object
date               object
price              object
asin               object
imageURL           object
imageURLHigh

## Merging both the datasets of review and metadata for  analysis

In [10]:
musical_instruments = pd.merge(reviews,metadata,on='asin')

In [12]:
#df.drop(['vote'],axis = 1, inplace = True)

In [11]:
musical_instruments.head()

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,...,feature,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details
0,5,90,False,"08 9, 2004",AXHY24HWOF184,0470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,4,2,True,"04 6, 2017",A29OWR79AM796H,0470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,5,NaN,True,"03 14, 2017",AUPWU27A7X5F6,0470536454,{'Format:': ' Paperback'},Amazon Customer,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,Five Stars,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,4,NaN,True,"02 14, 2017",A1N69A47D4JO6K,0470536454,{'Format:': ' Paperback'},Christopher Burnett,My girlfriend use quite often,Four Stars,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,5,NaN,True,"01 29, 2017",AHTIQUMVCGBFJ,0470536454,{'Format:': ' Paperback'},Amazon Customer,Arrived as described. Very happy.,Very happy.,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


- Shape of merge data(musical_instruments)
- Columns of merge data(musical_instruments)
- Data type of merge data(musical_instruments)

In [12]:
print('Shape of merge data:\n',musical_instruments.shape)
print('\n')
print('#'*100,'\n')
print('Columns of merge data:\n ',musical_instruments.columns)
print('\n')
print('#'*100,'\n')
print('Data type of merge data:\n',musical_instruments.info())

Shape of merge data:
 (1731703, 30)


#################################################################################################### 

Columns of merge data:
  Index(['overall', 'vote', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'image', 'category', 'tech1', 'description', 'fit', 'title', 'also_buy',
       'tech2', 'brand', 'feature', 'rank', 'also_view', 'main_cat',
       'similar_item', 'date', 'price', 'imageURL', 'imageURLHighRes',
       'details'],
      dtype='object')


#################################################################################################### 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1731703 entries, 0 to 1731702
Data columns (total 30 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   overall          int64 
 1   vote             object
 2   verified         bool  
 3   reviewTime       object
 4   reviewerID       object
 5 

In [13]:
musical_instruments_c=musical_instruments.copy()

### Checking for missing values in our merged data (musical_instruments).

In [14]:
musical_instruments_c.isnull().sum()

overall                  0
vote               1456961
verified                 0
reviewTime               0
reviewerID               0
asin                     0
style               937544
reviewerName           174
reviewText             971
summary                424
unixReviewTime           0
image              1699239
category                 0
tech1                    0
description              0
fit                      0
title                    0
also_buy                 0
tech2                    0
brand                    0
feature                  0
rank                     0
also_view                0
main_cat                 0
similar_item             0
date                     0
price                    0
imageURL                 0
imageURLHighRes          0
details               2529
dtype: int64

## Percentage of null values.

In [15]:
musical_instruments_c.isnull().sum()/len(musical_instruments)*100

overall             0.000000
vote               84.134577
verified            0.000000
reviewTime          0.000000
reviewerID          0.000000
asin                0.000000
style              54.140000
reviewerName        0.010048
reviewText          0.056072
summary             0.024485
unixReviewTime      0.000000
image              98.125314
category            0.000000
tech1               0.000000
description         0.000000
fit                 0.000000
title               0.000000
also_buy            0.000000
tech2               0.000000
brand               0.000000
feature             0.000000
rank                0.000000
also_view           0.000000
main_cat            0.000000
similar_item        0.000000
date                0.000000
price               0.000000
imageURL            0.000000
imageURLHighRes     0.000000
details             0.146041
dtype: float64

### Looking at the percentage of null values in the columns:- vote, image has high % of null value so i removed this columns from our merge data(musical_instruments)

In [17]:
musical_instruments_c.drop(['image','vote'],axis=1 ,inplace=True)
#musical_instruments_c = musical_instruments_c.drop(columns='vote')
musical_instruments_c.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,...,feature,rank,also_view,main_cat,similar_item,date,price,imageURL,imageURLHighRes,details
0,5,False,"08 9, 2004",AXHY24HWOF184,0470536454,{'Format:': ' Paperback'},Bendy,Crocheting for Dummies by Karen Manthey & Susa...,Terrific Book for Learning the Art of Crochet,1092009600,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,4,True,"04 6, 2017",A29OWR79AM796H,0470536454,{'Format:': ' Hardcover'},Amazon Customer,Very helpful...,Four Stars,1491436800,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,5,True,"03 14, 2017",AUPWU27A7X5F6,0470536454,{'Format:': ' Paperback'},Amazon Customer,EASY TO UNDERSTAND AND A PROMPT SERVICE TOO,Five Stars,1489449600,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
3,4,True,"02 14, 2017",A1N69A47D4JO6K,0470536454,{'Format:': ' Paperback'},Christopher Burnett,My girlfriend use quite often,Four Stars,1487030400,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,5,True,"01 29, 2017",AHTIQUMVCGBFJ,0470536454,{'Format:': ' Paperback'},Amazon Customer,Arrived as described. Very happy.,Very happy.,1485648000,...,[Dimensions: 9.2 in. h x 7.46 in. w x 0.78 in....,"[>#76,268 in Arts, Crafts &amp; Sewing (See To...","[1119287111, 156477998X, 0470045558, 190870724...","Arts, Crafts &amp; Sewing",,,,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [18]:
musical_instruments_c.shape

(1731703, 28)

### Removing all the null values from the containig columns of Null Values.

In [19]:
musical_instruments_c = musical_instruments_c.dropna()

In [20]:
musical_instruments_c.shape

(792613, 28)

In [21]:
print('Old data shape =>\n',musical_instruments.shape)
print('#'*100)
print('New data shape =>\n',musical_instruments_c.shape)

Old data shape =>
 (1731703, 30)
####################################################################################################
New data shape =>
 (792613, 28)


### Cheack Null value

In [22]:
musical_instruments_c.isnull().sum()

overall            0
verified           0
reviewTime         0
reviewerID         0
asin               0
style              0
reviewerName       0
reviewText         0
summary            0
unixReviewTime     0
category           0
tech1              0
description        0
fit                0
title              0
also_buy           0
tech2              0
brand              0
feature            0
rank               0
also_view          0
main_cat           0
similar_item       0
date               0
price              0
imageURL           0
imageURLHighRes    0
details            0
dtype: int64

### There are no missing value left

# Drop irrelevant columns

### What Is  Unix Time?

###### Unix time is a count of total seconds since a fixed time and date. It’s a date/time (or timestamp) format that looks different from the human-readable dates and times we’re used to. This is purely for efficiency reasons. It takes a lot less space to store a single number representing seconds than it does to store separate values for the year, month, hour, etc.

In [23]:
cols = ['fit', 'tech2','tech1', 'feature', 'details', 'date','imageURL', 'imageURLHighRes','unixReviewTime']

for i in cols:
    musical_instruments_c.drop([i],axis=1,inplace=True)

In [24]:
print('New data shape after drop some irrelevant:\n',musical_instruments_c.shape)

New data shape after drop some irrelevant:
 (792613, 19)


In [25]:
print('Columns of New data:\n ',musical_instruments_c.columns)

Columns of New data:
  Index(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style',
       'reviewerName', 'reviewText', 'summary', 'category', 'description',
       'title', 'also_buy', 'brand', 'rank', 'also_view', 'main_cat',
       'similar_item', 'price'],
      dtype='object')


In [26]:
musical_instruments_c.reset_index(drop=True,inplace=True)
musical_instruments_c.head(5)

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,category,description,title,also_buy,brand,rank,also_view,main_cat,similar_item,price
0,5,True,"09 21, 2016",A27HBIWF77N3H2,9792372326,{'style:': ' K10 Speaker'},Brian Hames,One of the best speakers on the market. This t...,Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
1,5,True,"09 10, 2016",AMKCR2ZIAZ7OI,9792372326,{'style:': ' K10 Speaker'},M W,I'm used to pretty high quality gear. My prim...,Very impressive,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
2,5,True,"09 4, 2016",A3TG7K64NML0DT,9792372326,{'style:': ' K10 Speaker'},W. K.,Great loudspeaker for home Karaoke. Love it.,Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
3,5,True,"08 16, 2016",ANJFHZEW2KB6Y,9792372326,{'style:': ' K10 Speaker'},Ma &#039;am,"Beautiful, crystal clear sound!",Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
4,5,False,"12 22, 2015",AVPFQX06IL5DW,9792372326,{'style:': ' K10 Speaker'},art hoff,I play in three bands Rock /Country /and Churc...,this system sounds great easy to set up lightw...,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99


### Reformat date time from raw form

In [27]:
musical_instruments_c["reviewTime"] = pd.to_datetime(musical_instruments_c["reviewTime"])

In [28]:
musical_instruments_c.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,category,description,title,also_buy,brand,rank,also_view,main_cat,similar_item,price
0,5,True,2016-09-21,A27HBIWF77N3H2,9792372326,{'style:': ' K10 Speaker'},Brian Hames,One of the best speakers on the market. This t...,Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
1,5,True,2016-09-10,AMKCR2ZIAZ7OI,9792372326,{'style:': ' K10 Speaker'},M W,I'm used to pretty high quality gear. My prim...,Very impressive,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
2,5,True,2016-09-04,A3TG7K64NML0DT,9792372326,{'style:': ' K10 Speaker'},W. K.,Great loudspeaker for home Karaoke. Love it.,Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
3,5,True,2016-08-16,ANJFHZEW2KB6Y,9792372326,{'style:': ' K10 Speaker'},Ma &#039;am,"Beautiful, crystal clear sound!",Five Stars,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99
4,5,False,2015-12-22,AVPFQX06IL5DW,9792372326,{'style:': ' K10 Speaker'},art hoff,I play in three bands Rock /Country /and Churc...,this system sounds great easy to set up lightw...,"[Musical Instruments, Live Sound &amp; Stage, ...","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",[],QSC,"[>#66,697 in Musical Instruments (See Top 100 ...","[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,,$125.99


In [29]:
musical_instruments_c = musical_instruments_c[['asin', 'summary', 'reviewText','description', 'title','overall','brand','rank','verified', 'reviewerID', 'reviewerName', 'reviewTime',
                    'price','style','category','also_buy','also_view','main_cat','similar_item']]

In [30]:
musical_instruments_c.head()

,asin,summary,reviewText,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item
0,9792372326,Five Stars,One of the best speakers on the market. This t...,"[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",5,QSC,"[>#66,697 in Musical Instruments (See Top 100 ...",True,A27HBIWF77N3H2,Brian Hames,2016-09-21,$125.99,{'style:': ' K10 Speaker'},"[Musical Instruments, Live Sound &amp; Stage, ...",[],"[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,
1,9792372326,Very impressive,I'm used to pretty high quality gear. My prim...,"[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",5,QSC,"[>#66,697 in Musical Instruments (See Top 100 ...",True,AMKCR2ZIAZ7OI,M W,2016-09-10,$125.99,{'style:': ' K10 Speaker'},"[Musical Instruments, Live Sound &amp; Stage, ...",[],"[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,
2,9792372326,Five Stars,Great loudspeaker for home Karaoke. Love it.,"[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",5,QSC,"[>#66,697 in Musical Instruments (See Top 100 ...",True,A3TG7K64NML0DT,W. K.,2016-09-04,$125.99,{'style:': ' K10 Speaker'},"[Musical Instruments, Live Sound &amp; Stage, ...",[],"[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,
3,9792372326,Five Stars,"Beautiful, crystal clear sound!","[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",5,QSC,"[>#66,697 in Musical Instruments (See Top 100 ...",True,ANJFHZEW2KB6Y,Ma &#039;am,2016-08-16,$125.99,{'style:': ' K10 Speaker'},"[Musical Instruments, Live Sound &amp; Stage, ...",[],"[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,
4,9792372326,this system sounds great easy to set up lightw...,I play in three bands Rock /Country /and Churc...,"[QSC K10 2-Way Powered Speaker - 1000 Watts, 1...","QSC K10 2-Way Powered Speaker - 1000 Watts, 1x10""",5,QSC,"[>#66,697 in Musical Instruments (See Top 100 ...",False,AVPFQX06IL5DW,art hoff,2015-12-22,$125.99,{'style:': ' K10 Speaker'},"[Musical Instruments, Live Sound &amp; Stage, ...",[],"[B06Y4BT184, B00CE0VLXO, B00NIE80A2, B000BKJPQ...",Musical Instruments,


In [31]:
musical_instruments_c.shape

(792613, 19)

### Check Duplicate value

In [32]:
#find duplicate rows across  columns
musical_instruments_c.duplicated(['asin','reviewText','reviewerName','reviewerID']).sum()

140352

In [33]:
musical_instruments_c.drop_duplicates(subset=['asin','reviewText','reviewerName','reviewerID'], keep='first',inplace=True)

In [34]:
#find duplicate rows across  columns
musical_instruments_c.duplicated(['asin','reviewText','reviewerName','reviewerID']).sum()

0

In [35]:
print('New data shape after drop duplicate value:\n',musical_instruments_c.shape)

New data shape after drop duplicate value:
 (652261, 19)


In [36]:
musical_instruments_c[['rank']]

,rank
0,"[>#66,697 in Musical Instruments (See Top 100 ..."
1,"[>#66,697 in Musical Instruments (See Top 100 ..."
2,"[>#66,697 in Musical Instruments (See Top 100 ..."
3,"[>#66,697 in Musical Instruments (See Top 100 ..."
4,"[>#66,697 in Musical Instruments (See Top 100 ..."
...,...
792608,"[>#12,010 in Musical Instruments (See Top 100 ..."
792609,"[>#88,955 in Musical Instruments (See Top 100 ..."
792610,"[>#88,955 in Musical Instruments (See Top 100 ..."
792611,"[>#75,132 in Musical Instruments (See Top 100 ..."


In [37]:
#for column 'rank' 
#remove uneccessary symbol like comma ',' and words
#convert the string into integer 
musical_instruments_c['rank'].replace(',','', regex=True, inplace=True)
musical_instruments_c['rank'] = musical_instruments_c['rank'].str.extract('(\d+)')

In [38]:
musical_instruments_c[['price']]

,price
0,$125.99
1,$125.99
2,$125.99
3,$125.99
4,$125.99
...,...
792608,$1.97
792609,$0.91
792610,$0.91
792611,$12.99


In [39]:
musical_instruments_c['price'].unique

<bound method Series.unique of 0         $125.99
1         $125.99
2         $125.99
3         $125.99
4         $125.99
           ...   
792608      $1.97
792609      $0.91
792610      $0.91
792611     $12.99
792612     $14.99
Name: price, Length: 652261, dtype: object>

In [40]:
#for column 'price'
#if 'price' don't start with $ or has '-' means null value, fillna with 'unknown'
#remove uneccessary symbol like comma ',' and '$'
#product_table.loc[product_table["price"].str[0] != "$", "price"] = "unknown"
#product_table.loc[product_table["price"].str.contains('-'), "price"] = "unknown"
musical_instruments_c['price'].replace("\$","", regex=True, inplace=True)
musical_instruments_c['price'].replace(',','', regex=True, inplace=True)

In [41]:
print("Number of reviews having empty string :",musical_instruments_c[musical_instruments_c['price']==''].shape[0])

Number of reviews having empty string : 89538


In [42]:
musical_instruments_c.isnull().sum()

asin                 0
summary              0
reviewText           0
description          0
title                0
overall              0
brand                0
rank            646919
verified             0
reviewerID           0
reviewerName         0
reviewTime           0
price                0
style                0
category             0
also_buy             0
also_view            0
main_cat             0
similar_item         0
dtype: int64

In [43]:
musical_instruments_c.dropna(inplace=True)

In [44]:
musical_instruments_c.isnull().sum()

asin            0
summary         0
reviewText      0
description     0
title           0
overall         0
brand           0
rank            0
verified        0
reviewerID      0
reviewerName    0
reviewTime      0
price           0
style           0
category        0
also_buy        0
also_view       0
main_cat        0
similar_item    0
dtype: int64

In [45]:
musical_instruments_c = musical_instruments_c[musical_instruments_c['price']!=''] 

In [46]:
print("Number of reviews having empty string :",musical_instruments_c[musical_instruments_c['price']==''].shape[0])

Number of reviews having empty string : 0


In [47]:
musical_instruments_c.head()

,asin,summary,reviewText,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item
24539,B0002BFNO0,Three Stars,what can you say about palstic grubs.,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,False,A3NTFZS442MKPG,Mark,2014-08-18,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24541,B0002BFNO0,Crappie lure,Leave this one at home if you have the others ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,1,Bass Assassin,390192,True,A2II5RVB13J0M5,Ignacio Ruben Garibay,2014-06-18,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24543,B0002BFNO0,Can't beat them,These are the best soft baits I have ever used...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AC29NDEY0MZQD,j harr1187,2014-04-01,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24545,B0002BFNO0,Crappie and bass,It's good for catching crappie and bass. The ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,True,A1YZIGL9TPCRD,THONG FANG,2013-05-28,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24547,B0002BFNO0,They were exactly as described and we are happ...,My hubby loves these for the lake fishing he d...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1UY5XOHNC97PY,Merryheart13,2018-03-06,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,


In [69]:
musical_instruments_c.shape

In [48]:
##changing data type
musical_instruments_c['rank']= musical_instruments_c['rank'].astype(int)
musical_instruments_c['price'] = musical_instruments_c['price'].apply(pd.to_numeric,errors='coerce')

In [49]:
musical_instruments_c.head(10)

,asin,summary,reviewText,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item
24539,B0002BFNO0,Three Stars,what can you say about palstic grubs.,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,False,A3NTFZS442MKPG,Mark,2014-08-18,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24541,B0002BFNO0,Crappie lure,Leave this one at home if you have the others ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,1,Bass Assassin,390192,True,A2II5RVB13J0M5,Ignacio Ruben Garibay,2014-06-18,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24543,B0002BFNO0,Can't beat them,These are the best soft baits I have ever used...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AC29NDEY0MZQD,j harr1187,2014-04-01,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24545,B0002BFNO0,Crappie and bass,It's good for catching crappie and bass. The ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,True,A1YZIGL9TPCRD,THONG FANG,2013-05-28,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24547,B0002BFNO0,They were exactly as described and we are happ...,My hubby loves these for the lake fishing he d...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1UY5XOHNC97PY,Merryheart13,2018-03-06,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24549,B0002BFNO0,Five Stars,Sweet,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1CGJNZGENWYXQ,Jr. Martinez,2017-02-13,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Salt and Pepp...","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24551,B0002BFNO0,Five Stars,Worked great on a lake.,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AXQLDXIF1N3EK,BYS,2016-12-11,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Alewife'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24553,B0002BFNO0,Great Drop Shot Lure.,"Great drop shot lure, catching me a lot of Per...",[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,A2IPZZX24IFCVQ,Alan. J. Reynolds,2016-04-01,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Salt and Pepp...","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24555,B0002BFNO0,My go-to lure,"I can truthfully say I have caught fish (bass,...",[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,ALJCWZX5HH5QN,Wedeliver2You,2015-09-24,5.74,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}","[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XD

In [50]:
musical_instruments_c.shape

(4366, 19)

In [51]:
print('Data type of merge data:\n',musical_instruments_c.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4366 entries, 24539 to 791583
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   asin          4366 non-null   object        
 1   summary       4366 non-null   object        
 2   reviewText    4366 non-null   object        
 3   description   4366 non-null   object        
 4   title         4366 non-null   object        
 5   overall       4366 non-null   int64         
 6   brand         4366 non-null   object        
 7   rank          4366 non-null   int32         
 8   verified      4366 non-null   bool          
 9   reviewerID    4366 non-null   object        
 10  reviewerName  4366 non-null   object        
 11  reviewTime    4366 non-null   datetime64[ns]
 12  price         4350 non-null   float64       
 13  style         4366 non-null   object        
 14  category      4366 non-null   object        
 15  also_buy      4366 non-null   ob

### Cleaning the style columns

In [52]:
musical_instruments_c[['style']]

,style
24539,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}"
24541,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}"
24543,"{'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}"
24545,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}"
24547,"{'Size:': ' 2-Inch', 'Color:': ' Black Shad'}"
...,...
791347,{'Color:': ' Purple'}
791348,{'Color:': ' Multicolor'}
791349,{'Color:': ' Multicolor'}
791582,{'Format:': ' CD-ROM'}


In [53]:
style= list(musical_instruments_c['style'].items())
style

[(24539, {'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}),
 (24541, {'Size:': ' 2-Inch', 'Color:': ' Black Shad'}),
 (24543, {'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}),
 (24545, {'Size:': ' 2-Inch', 'Color:': ' Black Shad'}),
 (24547, {'Size:': ' 2-Inch', 'Color:': ' Black Shad'}),
 (24549, {'Size:': ' 2-Inch', 'Color:': ' Salt and Pepper Silver'}),
 (24551, {'Size:': ' 2-Inch', 'Color:': ' Alewife'}),
 (24553, {'Size:': ' 2-Inch', 'Color:': ' Salt and Pepper Silver'}),
 (24555, {'Size:': ' 2-Inch', 'Color:': ' Black Shad'}),
 (24557, {'Size:': ' 2-Inch', 'Color:': ' Alewife'}),
 (24559, {'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}),
 (24561, {'Size:': ' 2-Inch', 'Color:': ' Salt and Pepper Silver'}),
 (24563, {'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}),
 (24565, {'Size:': ' 2-Inch', 'Color:': ' Crystal Shad'}),
 (252646, {'Item Display Length:': ' 21.00', 'Length:': ' 21.00'}),
 (252647, {'Item Display Length:': ' 22.00', 'Length:': ' 22.00'}),
 (252648, {'Item Display L

In [54]:
#musical_instruments_c.loc[musical_instruments_c["style"].str[0] != "$", "price"] = "unknown"
#musical_instruments_c.loc[musical_instruments_c["style"].str.contains('-'), "price"] = "unknown"
musical_instruments_c['style'].replace("size",'', regex=True, inplace=True)
musical_instruments_c['style'].replace([":","''"],'', regex=True, inplace=True)
#musical_instruments_c.fillna("unknown",inplace = True)

In [55]:
# Define function to clean dictionary string
def clean_dict_string(s):
    # Remove leading/trailing whitespace and curly braces
    s = s.str.strip('{}').strip()
    return s

In [56]:
musical_instruments_c['style'] = musical_instruments_c['style'].apply(lambda x: json.dumps(x)[1:-1])

In [57]:
musical_instruments_c['style'].replace(["Size","Color"],'', regex=True, inplace=True)
musical_instruments_c['style'].replace([":","''"],'', regex=True, inplace=True)
musical_instruments_c['style'].replace(r'[^a-zA-Z0-9\s]','', regex=True, inplace=True)

In [58]:
musical_instruments_c['style'].replace(r'\s+', ' ', regex=True, inplace=True)

In [59]:
print(list(musical_instruments_c['style']))

[' 2Inch Crystal Shad', ' 2Inch Black Shad', ' 2Inch Crystal Shad', ' 2Inch Black Shad', ' 2Inch Black Shad', ' 2Inch Salt and Pepper Silver', ' 2Inch Alewife', ' 2Inch Salt and Pepper Silver', ' 2Inch Black Shad', ' 2Inch Alewife', ' 2Inch Crystal Shad', ' 2Inch Salt and Pepper Silver', ' 2Inch Crystal Shad', ' 2Inch Crystal Shad', 'Item Display Length 2100 Length 2100', 'Item Display Length 2200 Length 2200', 'Item Display Length 2000 Length 2000', 'Item Display Length 2000 Length 2000', 'Item Display Length 2400 Length 2400', 'Item Display Length 2800 Length 2800', 'Item Display Length 2600 Length 2600', 'Item Display Length 2800 Length 2800', 'Item Display Length 2600 Length 2600', 'Item Display Length 2800 Length 2800', 'Item Display Length 2800 Length 2800', 'Item Display Length 2600 Length 2600', 'Item Display Length 2600 Length 2600', 'Item Display Length 2800 Length 2800', 'Item Display Length 2600 Length 2600', 'Item Display Length 2700 Length 2700', 'Item Display Length 2800

In [60]:
musical_instruments_c['style'].value_counts()

 Black                                  1111
 one size                                837
 One Green                               248
 3X128X108                               159
 One BLACK                               125
                                        ... 
Item Display Length 3200 Length 3200       1
 Red Red Purple                            1
 18                                        1
 8XLR Female                               1
 White 112 Large                           1
Name: style, Length: 196, dtype: int64

In [61]:
musical_instruments_c.head(10)

,asin,summary,reviewText,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item
24539,B0002BFNO0,Three Stars,what can you say about palstic grubs.,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,False,A3NTFZS442MKPG,Mark,2014-08-18,5.74,2Inch Crystal Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24541,B0002BFNO0,Crappie lure,Leave this one at home if you have the others ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,1,Bass Assassin,390192,True,A2II5RVB13J0M5,Ignacio Ruben Garibay,2014-06-18,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24543,B0002BFNO0,Can't beat them,These are the best soft baits I have ever used...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AC29NDEY0MZQD,j harr1187,2014-04-01,5.74,2Inch Crystal Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24545,B0002BFNO0,Crappie and bass,It's good for catching crappie and bass. The ...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,True,A1YZIGL9TPCRD,THONG FANG,2013-05-28,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24547,B0002BFNO0,They were exactly as described and we are happ...,My hubby loves these for the lake fishing he d...,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1UY5XOHNC97PY,Merryheart13,2018-03-06,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24549,B0002BFNO0,Five Stars,Sweet,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1CGJNZGENWYXQ,Jr. Martinez,2017-02-13,5.74,2Inch Salt and Pepper Silver,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24551,B0002BFNO0,Five Stars,Worked great on a lake.,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AXQLDXIF1N3EK,BYS,2016-12-11,5.74,2Inch Alewife,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24553,B0002BFNO0,Great Drop Shot Lure.,"Great drop shot lure, catching me a lot of Per...",[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,A2IPZZX24IFCVQ,Alan. J. Reynolds,2016-04-01,5.74,2Inch Salt and Pepper Silver,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24555,B0002BFNO0,My go-to lure,"I can truthfully say I have caught fish (bass,...",[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,ALJCWZX5HH5QN,Wedeliver2You,2015-09-24,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,
24557,B0002BFNO0,Five Stars,A OK,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AWFOZ9WJ6C7FR,Roger Flaugh,2015-08-30,5.74,2Inch Alewife,"[Mus

In [62]:
musical_instruments_c.shape

(4366, 19)

### Combining(concatenating) review text and review summary columns

In [63]:
musical_instruments_c['Review'] = musical_instruments_c['reviewText'] + musical_instruments_c['summary']
musical_instruments_c = musical_instruments_c.drop(['reviewText','summary'], axis=1)
musical_instruments_c.shape

(4366, 18)

In [64]:
musical_instruments_c.head(1)

,asin,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item,Review
24539,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,False,A3NTFZS442MKPG,Mark,2014-08-18,5.74,2Inch Crystal Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,what can you say about palstic grubs.Three Stars


### Data cleaning and removing special characters

In [65]:
def clean_text(text):
    # remove any non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # convert all text to lower case
    text = text.lower()
    # remove any URLs
    text = re.sub(r'http\S+', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    # remove any email addresses
    text = re.sub(r'\S+@\S+', '', text)
    # remove any numbers, digits, and hyphens
    text = re.sub(r'[0-9\-]+', '', text)
    #: Removes any punctuation marks from the string.
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    #Removes any newline characters.
    text = re.sub('\n', '', text)
    # remove any extra white space
    text = re.sub(r'\s+', ' ', text)
    # remove leading and trailing white space
    text = text.strip()
    return text

In [66]:
musical_instruments_c['Review'] = musical_instruments_c['Review'].apply(lambda x:clean_text(x))
musical_instruments_c[['Review']]

,Review
24539,what can you say about palstic grubsthree stars
24541,leave this one at home if you have the others ...
24543,these are the best soft baits i have ever used...
24545,its good for catching crappie and bass the wig...
24547,my hubby loves these for the lake fishing he d...
...,...
791347,closure strap tore within weekstrap tore
791348,nice i like itfive stars
791349,greatfive stars
791582,always wanted to get this product i finally di...


### Removing stop words

In [67]:
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS                     # Importing stop words from English language.
print('Number of stop words: %d' % len(spacy_stopwords))

Number of stop words: 326


In [68]:
print(spacy_stopwords)

{'if', 'n‘t', 'forty', 'both', 'less', 'may', 'with', 'their', 'yourself', 'beyond', 'get', 'just', 'across', 'indeed', 'seeming', 'per', 'three', 'them', 'only', 'could', 'one', 'upon', 'among', 'and', 'did', 'along', 'such', 'most', 'still', "n't", 'by', 'everything', 'also', "'re", 'becomes', 'or', 'can', 'serious', 'itself', 'from', 'do', 'eight', 'must', 'my', "'s", 'quite', 'call', 'of', 'thus', 'hundred', 'least', 'neither', 'enough', 'no', 'though', 'whereupon', 'herein', 'each', 'somehow', 'during', 'whole', 'too', 're', 'wherein', 'beforehand', 'eleven', 'any', 'every', 'herself', 'whence', 'before', 'his', 'many', 'ten', 'was', 'until', 'been', 'now', 'part', 'mostly', 'perhaps', 'him', 'towards', 'whom', 'at', 'hereby', 'often', 'should', 'everywhere', 'all', '’ll', '’s', 'same', 'moreover', 'formerly', '‘ll', 'yet', 'therefore', 'who', 'hereupon', 'someone', 'former', '‘ve', 'somewhere', 'others', 'our', 'few', 'nowhere', 'without', 'seemed', 'when', 'so', 'she', 'other', 

In [69]:
'five' in spacy_stopwords

True

In [70]:
'stars' in spacy_stopwords

False

In [71]:
excluding = ['against','not','don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't",
             'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 
             'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't",'shouldn', "shouldn't", 'wasn',
             "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't",'empty','enough','could','good','except','bad','five',
             'star','pretty','better','smooth','uneven']
stop_words = [words for words in spacy_stopwords if words not in excluding]

In [72]:
print('Number of stop words: %d' % len(stop_words))

Number of stop words: 319


In [73]:
musical_instruments_c['Review'] = musical_instruments_c['Review'].apply(lambda x: ' '.join([i for i in x.split() if i not in stop_words]))

In [74]:
def lemma(text):
    tokens = nlp(text.lower())
  #Lemmatization

    final=[]
    for word in tokens:
        final.append(word.lemma_) 
    text = " ".join(final)
    return text

In [75]:
musical_instruments_c['Review'] = musical_instruments_c['Review'].apply(lambda x:lemma(x))
musical_instruments_c['Review']

24539                               palstic grubsthree star
24541     leave home star rating simply not work color l...
24543     well soft bait catch killer crappie bass small...
24545     good catch crappie bass wiggly tail fishs atte...
24547     hubby love lake fish exactly describe happy pu...
                                ...                        
791347                    closure strap tore weekstrap tore
791348                                nice like itfive star
791349                                       greatfive star
791582    want product finally learn time thankswant lea...
791583    use bandinabox backing track write new song ea...
Name: Review, Length: 4366, dtype: object

# Sentiment distribution based on polarity scores

### TextBlob:- determing sentiments by creating polarity score of Review-Text

In [76]:
# Create a new column for sentiment polarity scores
musical_instruments_c["polarity"] = musical_instruments_c["Review"].apply(lambda x: TextBlob(x).sentiment.polarity)

In [77]:
musical_instruments_c["polarity"]

24539     0.000000
24541    -0.183333
24543     0.150000
24545     0.062500
24547     0.520000
            ...   
791347    0.000000
791348    0.600000
791349    0.000000
791582    0.000000
791583    0.379293
Name: polarity, Length: 4366, dtype: float64

In [78]:
def pos_neg(score):
    
    '''Returns the sentiment of the polarity'''
    
    if score < 0:
        sentiment_type = 'Negative'
    elif score == 0:
        sentiment_type = 'Neutral'
    else:
        sentiment_type = 'Positive'
        
    return sentiment_type

In [79]:
musical_instruments_c['Sentiment'] = musical_instruments_c['polarity'].apply(pos_neg)

In [80]:
musical_instruments_c.head(10)

,asin,description,title,overall,brand,rank,verified,reviewerID,reviewerName,reviewTime,price,style,category,also_buy,also_view,main_cat,similar_item,Review,polarity,Sentiment
24539,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,False,A3NTFZS442MKPG,Mark,2014-08-18,5.74,2Inch Crystal Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,palstic grubsthree star,0.000000,Neutral
24541,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,1,Bass Assassin,390192,True,A2II5RVB13J0M5,Ignacio Ruben Garibay,2014-06-18,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,leave home star rating simply not work color l...,-0.183333,Negative
24543,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AC29NDEY0MZQD,j harr1187,2014-04-01,5.74,2Inch Crystal Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,well soft bait catch killer crappie bass small...,0.150000,Positive
24545,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,3,Bass Assassin,390192,True,A1YZIGL9TPCRD,THONG FANG,2013-05-28,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,good catch crappie bass wiggly tail fishs atte...,0.062500,Positive
24547,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1UY5XOHNC97PY,Merryheart13,2018-03-06,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,hubby love lake fish exactly describe happy pu...,0.520000,Positive
24549,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,A1CGJNZGENWYXQ,Jr. Martinez,2017-02-13,5.74,2Inch Salt and Pepper Silver,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,sweetfive star,0.000000,Neutral
24551,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AXQLDXIF1N3EK,BYS,2016-12-11,5.74,2Inch Alewife,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,work great lakefive star,0.800000,Positive
24553,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,A2IPZZX24IFCVQ,Alan. J. Reynolds,2016-04-01,5.74,2Inch Salt and Pepper Silver,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,great drop shoot lure catch lot perch variety ...,0.800000,Positive
24555,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,False,ALJCWZX5HH5QN,Wedeliver2You,2015-09-24,5.74,2Inch Black Shad,"[Musical Instruments, Guitars]","[B005Z829YC, B0084EHLLK, B003ZZ81AS, B009A77KR...","[B008DR4XDM, B009M3TP3Y, B009A77KR0, B008DFGUK...",Sports &amp; Outdoors,,truthfully catch fish bass crappie bluegill ch...,0.175000,Positive
24557,B0002BFNO0,[The 2 inch Curly Shad has the same body desig...,Bass Assassin Curly Shads,5,Bass Assassin,390192,True,AWFOZ9WJ6C7FR,Roger Flaugh,2015-08-30,5.74,2Inch Alewife,"[Musical Instruments, Gui

In [81]:
musical_instruments_c.drop(['description','polarity'],axis=1,inplace=True)

In [82]:
print('New data shape after drop value:\n',musical_instruments_c.shape)

New data shape after drop value:
 (4366, 18)


In [83]:
musical_instruments_data=musical_instruments_c.copy()

In [84]:
musical_instruments_data.to_csv('musical_instruments_data.csv')

In [85]:
musical_instruments_data.columns

Index(['asin', 'title', 'overall', 'brand', 'rank', 'verified', 'reviewerID',
       'reviewerName', 'reviewTime', 'price', 'style', 'category', 'also_buy',
       'also_view', 'main_cat', 'similar_item', 'Review', 'Sentiment'],
      dtype='object')